# **PROJECT 4**

# 🔥 **Detecting Fake News Using Machine Learning 🚀**

### Fake news is a growing problem in today's society, and it can have serious consequences, including influencing public opinion and even shaping policy decisions. In this project, we aim to build a machine learning model that can accurately identify whether a given news article is fake or not.

---

Dataset used - https://www.kaggle.com/fake-news/data

Dataset Description

train.csv: A full training dataset with the following attributes:

* id: unique id for a news article
* title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable
  * 1: FAKE
  * 0: TRUE

train.csv: The full training dataset, with the labels removed

---


### **Set the dependencies**

In [1]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import inspect
from api_keys import postgres_p
import matplotlib.pyplot as plt
import re 
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import itertools
import pickle
import winsound

---


### **Retrieving Text Data from PostgreSQL and Loading into a Pandas DataFrame**

First, the code downloads and loads all stop words from the NLTK corpus. Then, it establishes a connection to a PostgreSQL database named "Project_4" using the psycopg2 library. A SQL query is executed to retrieve data from two tables named "article_id" and "text_corpus" in the database. The data is limited to the first 1000 rows using the LIMIT keyword. The results are stored in a Pandas DataFrame named "news_dataset". Finally, the database connection is closed using the close() method.

In [2]:
# Download and load all stop words
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonow\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# check if Stopwords loaded in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

---

### **Data Pre-processing and Analysis**


In [4]:
# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(database="Project_4", user="postgres", password=postgres_p) #host="your_host_address", port="your_port_number"

In [5]:
# SQL query to retrieve the data - Limit to 1000 records for testing and evaluation
query = "SELECT a.article_id, a.article_label, t.text_corpus FROM article_id a  JOIN text_corpus t ON a.article_id = t.article_id LIMIT 5000"

In [6]:
# Execute the query and store the results in a Pandas DataFrame
news_dataset = pd.read_sql_query(query, conn)

In [7]:
# Close the database connection
conn.close()

In [8]:
# Check dataset
news_dataset.head()

,article_id,article_label,text_corpus
0,0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,1,Consortiumnews.com Why the Truth Might Get You...
3,3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,1,Howard Portnoy Iranian woman jailed for fictio...


---

### **Text Preprocessing with Stemming, Stop Word Removal, and Lowercasing**

In the following code, the data and label are separated into two variables, X and Y, respectively, from the previously loaded dataset. Then, a function is defined to perform stemming on the text data. 

Stemming is the process of reducing words to their base or root form. The PorterStemmer class from the nltk package is used for stemming. 

The function removes non-alphabetic characters and converts all remaining letters to lowercase. It splits the text into individual words, removes stop words, and applies stemming to each word using the PorterStemmer. Finally, it joins all the words back into a string. 

The apply() method is used to apply the stemming() function to each row of the text_corpus column of the dataset, and the results are stored back in X. 

Lastly, a beep sound is played to signal that the process has completed.

In [9]:
# Separate the data and label into two variables
X = news_dataset['text_corpus']
Y = news_dataset['article_label']

In [10]:
# Define a function for stemming the content
port_stem = PorterStemmer()
def stemming(content):
    # Pick all alphabet characters - lowercase and uppercase...all others such as numbers and punctuations will be removed. Numbers or punctuations will be replaced by a whitespace
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    # Converting all letters to lowercase 
    stemmed_content = stemmed_content.lower()
    # Converting all to a splitted case or a list
    stemmed_content = stemmed_content.split()
    # Applying stemming, so we get the root words wherever possible + remove stopwords as well
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Join all the words in final content
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [11]:
# Apply stemming to the text_corpus column
X = X.apply(stemming)

In [ ]:
# Play a sound to let you know its done 
duration = 2000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [ ]:
# Print the X and Y variables
print(X)
print(Y)

---

### **Transforming Text Data with CountVectorizer**

In the following code, a single instance of CountVectorizer is created to convert text data into numerical form. The stop_words parameter is set to 'english' to remove common stop words from the text. The fit() method is called on the vectorizer object to learn the vocabulary of the text data. The transform() method is called on the X variable to convert the text data into a sparse matrix of numerical features. The resulting matrix is stored in X_transformed.

The pickle module is used to save the vectorizer object to a file named "tfidfvect2.pkl" in the "Pickles" directory. The saved object can be loaded and used later for transforming new text data into numerical features.

The print() function is used to display the vectorizer object and the transformed X data. The transformed X data is displayed as a sparse matrix representation.

In [ ]:
# Create a single instance of CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

In [ ]:
# Fit the vectorizer on the text data
vectorizer.fit(X)

In [ ]:
# Transform the data into numerical features
X_transformed = vectorizer.transform(X)

In [ ]:
# Save the vectorizer object to a file using pickle
pickle.dump(vectorizer, open('../Pickles/tfidfvect2.pkl', 'wb'))

In [ ]:
# Load the saved object and print it to check if it is loaded correctly
TEST_model = pickle.load(open('../Pickles/tfidfvect2.pkl', 'rb'))
print(TEST_model)

In [ ]:
# Print the transformed data
print(X_transformed)

---

### **CLASSIFICATION MODEL : LOGISTIC REGRESSION CLASSIFIER**

### Splitting the data into test and train datasets

The data is split into training and testing sets using the train_test_split function. The test_size parameter is set to 0.18, meaning that 18% of the data will be used for testing, and the remaining 82% for training. The random_state parameter is set to 42 to ensure reproducibility of the results.

In [ ]:
# Splitting the data into test and train datasets
X_train_transformed, X_test, Y_train, Y_test = train_test_split(X_transformed, Y, test_size=0.18, random_state=42)

A parameter grid is defined for hyperparameter tuning, including regularization strength C, regularization penalty type (l1 or l2), and solver algorithm (liblinear).

In [ ]:
# Define the parameter grid for Logistic Regression
logreg_param_grid = {
    'C': np.logspace(-4, 4, 20),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

In [ ]:
# Initialize the model
logreg = LogisticRegression()

In [ ]:
# Create a single instance of CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

GridSearchCV is used to find the best set of hyperparameters from the parameter grid. Cross-Validation with 5-folds is used to evaluate each set of hyperparameters.

In [ ]:
# Perform GridSearchCV for Logistic Regression
logreg_grid_search = GridSearchCV(logreg, logreg_param_grid, cv=5, verbose=0)
logreg_grid_search.fit(X_transformed, Y)

The best hyperparameters found by Grid Search are stored in best_logreg_params and are then used to train the final Logistic Regression model, best_logreg_model.

In [ ]:

# Get the best parameters
best_logreg_params = logreg_grid_search.best_params_

In [ ]:
# Train the model with best parameters
best_logreg_model = LogisticRegression(**best_logreg_params)

The trained model is used to predict the outcomes on the test set, and the prediction accuracy is calculated as the percentage of correct predictions.

In [ ]:
# Calculate the accuracies and record the changes
logreg_accuracies = logreg_grid_search.cv_results_['mean_test_score']

In [ ]:
best_logreg_model.fit(X_train_transformed, Y_train)

The results are displayed, including the accuracy, classification report, and confusion matrix.

In [ ]:
y_pred1 = best_logreg_model.predict(X_test)
accuracy1 = np.mean(y_pred1 == Y_test) * 100

print("Logistic Regression Model Results")
print("----------------------------------")
print("Prediction accuracy: {:.2f}%".format(accuracy1))
print("\nClassification Report:")
print("--------------------------------------------")
print(classification_report(Y_test, y_pred1))
print("Confusion Matrix:")
print(confusion_matrix(Y_test, y_pred1))
print("\n")

A plot is created to visualize the accuracy of different sets of hyperparameters during the Grid Search process. 

In [ ]:
# Plot changes
plt.figure(figsize=(12, 6))
plt.plot(logreg_accuracies, label="Logistic Regression", linestyle="-", marker="o")
plt.xlabel("Parameter Set")
plt.ylabel("Accuracy")
plt.title("Logistic Regression Model Optimization")
plt.legend()
plt.savefig("logreg_model_optimization.png")
plt.show()

The trained model is used to predict the outcomes on the training and test sets, and the prediction accuracy is calculated using the accuracy_score function from the sklearn.metrics module. 

In [ ]:
# Accuracy Score on Training Data
X_train_prediction = best_logreg_model.predict(X_train_transformed)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score on the training data: ',training_data_accuracy)

# Accuracy Score on Test Data
X_test_prediction = best_logreg_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Accuracy score on the test data: ',test_data_accuracy)

The trained Logistic Regression model is saved as a pickle file

In [ ]:
pickle.dump(best_logreg_model, open('../Pickles/logisticreg_model.pkl', 'wb'))

---

### **CLASSIFICATION MODEL : PASSIVE AGGRESSIVE CLASSIFIER**

Passive Aggressive Classifier works by responding as passive for correct classifications and responding as aggressive for any miscalculation.

A function called stemming is defined to preprocess and stem the input text. The function takes the input text, removes non-alphabetic characters, converts the text to lowercase, tokenizes it, removes stopwords, and applies stemming using the PorterStemmer algorithm.

In [ ]:
# Create a function to preprocess and stem the text
def stemming(text):
    ps = PorterStemmer()
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    return review


The stemming function is applied to each element of X. The preprocessed data is then transformed using the vectorizer, and the dataset is split into training and testing sets.

In [ ]:
# Apply stemming to each text in the array
X_preprocessed = [stemming(text) for text in X]

In [ ]:
# Fit and transform the preprocessed data
X_transformed = vectorizer.fit_transform(X_preprocessed)

In [ ]:
# Splitting dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_transformed, Y, test_size=0.33, random_state=42)

A parameter grid is defined for hyperparameter tuning, including regularization strength C, and loss function type (hinge or squared_hinge). GridSearchCV is used to find the best set of hyperparameters from the parameter grid.

In [ ]:
# Define the parameter grid for Passive Aggressive Classifier
pac_param_grid = {
    'C': np.logspace(-4, 4, 20),
    'loss': ['hinge', 'squared_hinge']
}

In [ ]:
# Initialize the model
pac = PassiveAggressiveClassifier()

In [ ]:
# Perform GridSearchCV for Passive Aggressive Classifier
pac_grid_search = GridSearchCV(pac, pac_param_grid, cv=5, verbose=0)
pac_grid_search.fit(X_transformed, Y)

In [ ]:

# Get the best parameters
best_pac_params = pac_grid_search.best_params_

In [ ]:
# Train the model with best parameters
best_pac_model = PassiveAggressiveClassifier(**best_pac_params)


In [ ]:
# Calculate the accuracies and record the changes
pac_accuracies = pac_grid_search.cv_results_['mean_test_score']

In [ ]:
best_pac_model.fit(X_train, Y_train)

The results are displayed, including the accuracy, classification report, and confusion matrix.

A plot is created to visualize the accuracy of different sets of hyperparameters during the Grid Search process. 

In [ ]:
# Plot changes
plt.figure(figsize=(12, 6))
plt.plot(pac_accuracies, label="Passive Aggressive Classifier", linestyle="--", marker="x")
plt.xlabel("Parameter Set")
plt.ylabel("Accuracy")
plt.title("Passive Aggressive Model Optimization")
plt.legend()
plt.savefig("model_optimization.png")
plt.show()

In [ ]:
# Passive Aggressive Classifier
y_pred2 = best_pac_model.predict(X_test)
accuracy2 = np.mean(y_pred2 == Y_test) * 100

print("Passive Aggressive Classifier Model Results")
print("--------------------------------------------")
print("Prediction accuracy: {:.2f}%".format(accuracy2))
print("\nClassification Report:")
print("--------------------------------------------")
print(classification_report(Y_test, y_pred2))
print("Confusion Matrix:")
print(confusion_matrix(Y_test, y_pred2))

In [ ]:
# Making prediction on test set
test_pred = best_pac_model.predict(X_test)

The trained TfidfVectorizer instance and the Passive Aggressive Classifier model are saved as pickle files

In [ ]:
# Save the vectorizer
pickle.dump(vectorizer, open('../Pickles/tfidf_vectorizer.pkl', 'wb'))

In [ ]:
# Save the model
pickle.dump(best_pac_model, open('../Pickles/passive_aggressive_model.pkl', 'wb'))

---

## Testing the two models

In [ ]:
# Preprocess and vectorize the input data
X_preprocessed = [stemming(text) for text in X]


In [ ]:
# Splitting dataset into train and test sets
X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(X_preprocessed, Y, test_size=0.18, random_state=42)



In [ ]:
# Fit the vectorizer on the training data and transform both the training and test data
X_train = vectorizer.fit_transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)

In [ ]:
# Retrain the Logistic Regression model with the best parameters
best_logreg_model = LogisticRegression(**best_logreg_params)
best_logreg_model.fit(X_train, Y_train)

In [ ]:
# Test Logistic Regression Model
y_pred = best_logreg_model.predict(X_test)
accuracy = np.mean(y_pred == Y_test) * 100
print("Logistic Regression Model Prediction Accuracy: {:.2f}%".format(accuracy))



In [ ]:
# Retrain the Passive Aggressive Classifier model with the best parameters
best_pac_model = PassiveAggressiveClassifier(**best_pac_params)
best_pac_model.fit(X_train, Y_train)

In [ ]:
# Test Passive Aggressive Classifier Model
y2_pred = best_pac_model.predict(X_test)
accuracy = np.mean(y2_pred == Y_test) * 100
print("Passive Aggressive Classifier Model Prediction Accuracy: {:.2f}%".format(accuracy))


In [ ]:
# Plot both models
plt.figure(figsize=(12, 6))
plt.plot(logreg_accuracies, label="Logistic Regression", linestyle="-", marker="o")
plt.plot(pac_accuracies, label="Passive Aggressive Classifier", linestyle="--", marker="x")
plt.xlabel("Parameter Set")
plt.ylabel("Accuracy")
plt.title("Model Optimization")
plt.legend()
plt.savefig("model_optimization.png")
plt.show()